In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from PositionalEmbedding3D import PositionalEmbedding3D

In [2]:
import torch
import torch.nn as nn

class MinecraftSequencePredict(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_encoder_layers, num_decoder_layers, src_shape: tuple, tgt_shape: tuple, tgt_offset: tuple, device):
        super(MinecraftSequencePredict, self).__init__()

        # Define the embedding layer
        self.block_embedding = nn.Embedding(vocab_size, d_model, device=device)

        self.pos_embedding = PositionalEmbedding3D(d_model, src_shape, tgt_shape, tgt_offset, device=device)
        # Define the transformer model
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            batch_first=True,
            device=device
        )

        # Define the output layer
        self.fc = nn.Linear(d_model, vocab_size,device=device)

    def forward(self, src, tgt):
        print(f'sizes:\nsrc: {src.shape}\ntgt: {tgt.shape}\n')
        
        src = self.block_embedding(src)
        tgt = self.block_embedding(tgt)
        
        print(f'sizes after block embeddings:\nsrc: {src.shape}\ntgt: {tgt.shape}\n')
        
        src = self.pos_embedding(src, True)
        tgt = self.pos_embedding(tgt, False)
        
        print(f'sizes after positional embeddings:\nsrc: {src.shape}\ntgt: {tgt.shape}\n')
        
        output = self.transformer(src, tgt)
        
        print(f'output shape from transformer:\n{output.shape}\n')

        # You may want to reshape or slice the output based on your specific task
        output = self.fc(output)
        
        print(f'output shape after linear layer:\n{output.shape}\n')

        return output

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Example usage
vocab_size = 255  # Adjust based on your vocabulary size
d_model = 768
nhead = 8
num_encoder_layers = 6
num_decoder_layers = 6

batch_size = 32
src_shape  = (5,5,5)
tgt_shape  = (1,5,5)
tgt_offset = (5,0,0)


# Set the seed for reproducibility (optional)
torch.manual_seed(42)

# Create a 5x5x5 tensor with random integers from 0 to 255
src = torch.randint(0, vocab_size - 1, size=src_shape, dtype=torch.int).view(-1).unsqueeze(0).expand(batch_size,-1).to(device)
tgt = torch.cat([torch.tensor([vocab_size-1]).view(-1),torch.randint(0, vocab_size - 1, size=tgt_shape, dtype=torch.int).view(-1)]).unsqueeze(0).expand(batch_size,-1).to(device)
tgt_sos = torch.tensor([vocab_size-1]).unsqueeze(0)
# print(src.device)
# print(tgt.device)


model = MinecraftSequencePredict(vocab_size, d_model, nhead, num_encoder_layers, num_decoder_layers, src_shape, tgt_shape, tgt_offset, device).to(device)

output = model(src, tgt)
# print(model.eval)
# torch.argmax(output)

sizes:
src: torch.Size([32, 125])
tgt: torch.Size([32, 26])

sizes after block embeddings:
src: torch.Size([32, 125, 768])
tgt: torch.Size([32, 26, 768])

cuda:0
cuda:0
cuda:0
positional embedding shape:
torch.Size([125, 768])
x shape:
torch.Size([32, 125, 768])
cuda:0
cuda:0
cuda:0
positional embedding shape:
torch.Size([26, 768])
x shape:
torch.Size([32, 26, 768])
sizes after positional embeddings:
src: torch.Size([32, 125, 768])
tgt: torch.Size([32, 26, 768])

output shape from transformer:
torch.Size([32, 26, 768])

output shape after linear layer:
torch.Size([32, 26, 255])



In [10]:
# Example usage
vocab_size = 255  # Adjust based on your vocabulary size
d_model = 768
nhead = 8
num_encoder_layers = 6
num_decoder_layers = 6

src_shape  = (5,5,5)
tgt_shape  = (1,5,5)
tgt_offset = (5,0,0)


# Set the seed for reproducibility (optional)
torch.manual_seed(42)

# Create a 5x5x5 tensor with random integers from 0 to 255
src = torch.randint(0, vocab_size - 1, size=src_shape, dtype=torch.int).ravel().unsqueeze(0)
tgt = torch.cat([torch.tensor([vocab_size-1]),torch.randint(0, vocab_size - 1, size=tgt_shape, dtype=torch.int).ravel()]).unsqueeze(0)

print(src)
print(tgt)

tensor([[170,  67, 148, 136, 200, 159, 242, 104,  24, 201,  84, 236,  94, 110,
         185, 182,  31, 175, 217, 132, 223, 102,   9,  79, 247,  61,   1,  97,
         143, 231, 142, 112, 109,  23, 137, 120, 155, 250, 188, 182, 122,  64,
          93, 183,  89, 201, 177,  24, 111,  55, 156,  11, 232,  51, 170, 189,
         157,  62, 202, 130,  86, 232,   2,  16,  58, 107,  37,   6,  37, 181,
         131,  15, 190, 141,  22, 131,  73, 203,  92, 157, 132, 179, 220, 117,
         138,  78, 183,   2, 133,  81,  73, 209,  77, 205, 157,  37,  15, 135,
          59, 120, 206, 205,  11, 131,  65, 187,  17,  73,  17, 242, 131, 106,
         253, 195, 246,  77, 172, 119, 155,  46, 167, 240, 159, 242, 202]],
       dtype=torch.int32)
tensor([[254,  77,  41, 252, 145, 107, 213, 136, 208, 138, 136, 154, 176, 248,
          12,  22,   3, 242,  17, 129,  85,  62,  78, 212,  10, 175]])


In [4]:
import numpy as np

# position = torch.arange(0, x.size(1)).unsqueeze(0)
shape = (4,4,4)

array_3d = np.empty(shape, dtype=object)

# Fill in the array with tuples of indices
for i in range(shape[0]):
    for j in range(shape[1]):
        for k in range(shape[2]):
            array_3d[i, j, k] = (i, j, k)
array_3d = array_3d.ravel()
print(array_3d)
arr = [i[0] for _, i in np.ndenumerate(array_3d)]
arr.insert(0, 12)
print(type(arr))

[(0, 0, 0) (0, 0, 1) (0, 0, 2) (0, 0, 3) (0, 1, 0) (0, 1, 1) (0, 1, 2)
 (0, 1, 3) (0, 2, 0) (0, 2, 1) (0, 2, 2) (0, 2, 3) (0, 3, 0) (0, 3, 1)
 (0, 3, 2) (0, 3, 3) (1, 0, 0) (1, 0, 1) (1, 0, 2) (1, 0, 3) (1, 1, 0)
 (1, 1, 1) (1, 1, 2) (1, 1, 3) (1, 2, 0) (1, 2, 1) (1, 2, 2) (1, 2, 3)
 (1, 3, 0) (1, 3, 1) (1, 3, 2) (1, 3, 3) (2, 0, 0) (2, 0, 1) (2, 0, 2)
 (2, 0, 3) (2, 1, 0) (2, 1, 1) (2, 1, 2) (2, 1, 3) (2, 2, 0) (2, 2, 1)
 (2, 2, 2) (2, 2, 3) (2, 3, 0) (2, 3, 1) (2, 3, 2) (2, 3, 3) (3, 0, 0)
 (3, 0, 1) (3, 0, 2) (3, 0, 3) (3, 1, 0) (3, 1, 1) (3, 1, 2) (3, 1, 3)
 (3, 2, 0) (3, 2, 1) (3, 2, 2) (3, 2, 3) (3, 3, 0) (3, 3, 1) (3, 3, 2)
 (3, 3, 3)]
<class 'list'>


In [8]:
src_shape = (5,5,5)
src_positions = np.empty(src_shape, dtype=object)

for i in range(src_shape[0]):
            for j in range(src_shape[1]):
                for k in range(src_shape[2]):
                    src_positions[i, j, k] = (i, j, k)
                    
# src_positions = torch.from_numpy(dtype=np.object_)

TypeError: empty() received an invalid combination of arguments - got (tuple, dtype=type), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
